Real-World Graph Application: LinkedIn's "People You May Know"
--------------------------------------------------------------

### The Problem: Connecting Professionals

LinkedIn's primary goal is to connect the world's professionals to make them more productive and successful. A crucial part of this is helping users discover relevant connections. The "People You May Know" (PYMK) feature is designed to suggest new connections to users.

### Why Graphs are the Perfect Fit

Imagine LinkedIn's user base. Each user is an individual entity, and their connections (1st-degree, 2nd-degree, etc.) form relationships. This naturally maps to a **graph data structure**:

*   **Nodes (Vertices):** Each LinkedIn user is a node.
    
*   **Edges:** A connection between two users (e.g., if User A is connected to User B) is an edge. Since connections on LinkedIn are usually mutual, this would be an **undirected graph**. If we considered "following" relationships, it would become a directed graph. For PYMK, we primarily focus on mutual connections.
    
*   **Unweighted/Weighted:** For the basic PYMK, the initial problem can be thought of as unweighted (a connection is just a connection). However, in a more advanced system, edges might be weighted by strength of connection, common groups, shared work history, etc., to prioritize suggestions.
    

The problem of finding "people you may know" is fundamentally about finding nodes that are "close" to you in the graph but not yet directly connected. This is where graph algorithms shine.

### Graph Construction and Data Structure Choice

LinkedIn deals with hundreds of millions of users and billions of connections. This is a massive graph!

Given the scale:

1.  **Number of Vertices (V):** Very Large (hundreds of millions).
    
2.  **Number of Edges (E):** Extremely Large (billions).
    
3.  **Sparsity:** The graph is **sparse**. Even though there are billions of edges, E is _much_ less than V2. No single user is connected to every other user.
    

**Decision: Adjacency List is the ONLY practical choice.**

*   **Adjacency Matrix (Why it's ruled out):** An O(V2) adjacency matrix would be astronomically large. If V=500 million, V2 is 2.5times10**17. Even storing booleans for this would require petabytes of memory, which is infeasible.
    
*   **Adjacency List (Why it's chosen):** An adjacency list uses O(V+E) space. For a sparse graph like LinkedIn's, this is vastly more efficient. Each user (node) stores a list (or set) of their direct connections. This allows for efficient traversal of neighbors.

In [1]:
from collections import defaultdict

# In a real system, user_id would be a unique identifier (e.g., long integer)
# This is a simplified representation of LinkedIn's core connections graph.
# 'connections' would be stored in a distributed database like Neo4j (a graph database)
# or sharded across relational/NoSQL databases with specialized graph processing layers.

user_connections = defaultdict(set) # Using a set for O(1) average time 'in' check and no duplicates

def add_connection(user1_id, user2_id):
    """Adds a mutual connection between two users."""
    user_connections[user1_id].add(user2_id)
    user_connections[user2_id].add(user1_id) # Undirected graph

# --- Simulate some LinkedIn connections ---
add_connection("Alice", "Bob")
add_connection("Alice", "Charlie")
add_connection("Bob", "David")
add_connection("Bob", "Eve")
add_connection("Charlie", "Frank")
add_connection("David", "Grace")
add_connection("Eve", "Heidi")
add_connection("Frank", "Heidi")
add_connection("Frank", "Bob") # Bob is already connected to Frank (implicitly through Frank->Bob)

print("Simplified LinkedIn Connections Graph:")
for user, connections in user_connections.items():
    print(f"{user}: {connections}")

# Example: Alice's connections: {'Bob', 'Charlie'}
# Bob's connections: {'Alice', 'David', 'Eve', 'Frank'}

Simplified LinkedIn Connections Graph:
Alice: {'Bob', 'Charlie'}
Bob: {'Frank', 'Alice', 'David', 'Eve'}
Charlie: {'Frank', 'Alice'}
David: {'Bob', 'Grace'}
Eve: {'Bob', 'Heidi'}
Frank: {'Bob', 'Heidi', 'Charlie'}
Grace: {'David'}
Heidi: {'Frank', 'Eve'}


### Graph Algorithms in Action for PYMK

The core idea behind PYMK is to find "friends of friends" who you are not already connected to. This is where both BFS and DFS concepts, or variations of them, become crucial.

#### Application 1: Basic "Friends of Friends" (BFS-like approach)

The simplest approach for PYMK is to find users who are 2 degrees away from you but not 1 degree away. This is a perfect fit for a Breadth-First Search (BFS) concept.

**How it works:**

1.  Start at the current user (e.g., "Alice").
    
2.  Find all of Alice's 1st-degree connections (e.g., Bob, Charlie).
    
3.  For each of these 1st-degree connections, find _their_ connections. These are Alice's 2nd-degree connections (e.g., David, Eve, Frank, Heidi).
    
4.  Filter out any 2nd-degree connections who are already 1st-degree connections of Alice, or are Alice herself.
    
5.  The remaining users are potential "People You May Know."

In [17]:
#Simplified Python Code for PYMK (BFS-like, conceptual for production):
from collections import deque

def get_people_you_may_know_basic(current_user, connections_graph, max_depth=2):
    """
    Finds potential connections up to a certain depth (e.g., 2 for friends-of-friends).
    This is a simplified BFS that tracks distance.
    """
    suggestions = set()
    visited = {current_user} # Keep track of visited users to avoid cycles and self-suggestions
    queue = deque([(current_user, 0)]) # (user_id, distance_from_source)

    print("current_user:",current_user)
    print("max_depth:",max_depth)
    print("connections_graph:",connections_graph)
    print("suggestions:",suggestions)
    print("visited:",visited)
    print("queue:",queue)
    
    print("--------------------------------")

    while queue:
        print("starting queue:", queue)
        user, distance = queue.popleft()
        print("queue after pop:",queue)

        print("user:",user)
        print("distance:",distance)
        if distance < max_depth: # Explore up to max_depth
            neighbors = connections_graph.get(user,[])
            print(f"Neightbors for the user {user}:{neighbors}")
            for neighbor in neighbors: # Use .get() for users with no connections
                print("neighbor:",neighbor)
                if neighbor not in visited:
                    visited.add(neighbor)
                    print("visited:",visited)
                    queue.append((neighbor, distance + 1))
                    if distance + 1 == max_depth: # If at the target depth for suggestions
                        suggestions.add(neighbor)
                        print("suggsetions:",suggestions)

    # Filter out direct connections and self (already handled by visited set if start at depth 0,
    # but important for clarity when suggestions are added at a specific depth).
    # In this specific BFS, if max_depth=2, direct connections are never added to suggestions.
    direct_connections = connections_graph.get(current_user, set())
    final_suggestions = suggestions - direct_connections - {current_user}

    return list(final_suggestions)

# --- Test the basic PYMK ---
print(f"\nPeople Alice may know (basic): {get_people_you_may_know_basic('Alice', user_connections)}")
# Expected (roughly): David, Eve, Frank, Heidi (excluding Bob, Charlie who are direct connections)
# Note: Frank is a direct connection of Bob, and Bob is direct connection of Alice. So Frank is a friend of friend.
# Let's trace Alice:
# Alice -> Bob (1st degree)
# Alice -> Charlie (1st degree)
# From Bob: David, Eve, Frank (2nd degree from Alice)
# From Charlie: Frank (2nd degree from Alice)
# Suggestions = {David, Eve, Frank}
# Alice's direct connections = {Bob, Charlie}
# Final suggestions = {David, Eve, Frank} (This is correct)

print(f"People David may know (basic): {get_people_you_may_know_basic('David', user_connections)}")
# David -> Bob (1st degree)
# David -> Grace (1st degree)
# From Bob: Alice, Charlie, Eve, Frank (2nd degree)
# From Grace: None in this small graph
# Filter out Bob. Result should be {'Alice', 'Charlie', 'Eve', 'Frank'}

current_user: Alice
max_depth: 2
connections_graph: defaultdict(<class 'set'>, {'Alice': {'Bob', 'Charlie'}, 'Bob': {'Frank', 'Alice', 'David', 'Eve'}, 'Charlie': {'Frank', 'Alice'}, 'David': {'Bob', 'Grace'}, 'Eve': {'Bob', 'Heidi'}, 'Frank': {'Bob', 'Heidi', 'Charlie'}, 'Grace': {'David'}, 'Heidi': {'Frank', 'Eve'}})
suggestions: set()
visited: {'Alice'}
queue: deque([('Alice', 0)])
--------------------------------
starting queue: deque([('Alice', 0)])
queue after pop: deque([])
user: Alice
distance: 0
Neightbors for the user Alice:{'Bob', 'Charlie'}
neighbor: Bob
visited: {'Bob', 'Alice'}
neighbor: Charlie
visited: {'Bob', 'Alice', 'Charlie'}
starting queue: deque([('Bob', 1), ('Charlie', 1)])
queue after pop: deque([('Charlie', 1)])
user: Bob
distance: 1
Neightbors for the user Bob:{'Frank', 'Alice', 'David', 'Eve'}
neighbor: Frank
visited: {'Bob', 'Frank', 'Alice', 'Charlie'}
suggsetions: {'Frank'}
neighbor: Alice
neighbor: David
visited: {'Bob', 'David', 'Alice', 'Frank', 'Charli

**Why BFS-like?** The "friends of friends" concept inherently seeks the shortest path (in terms of degrees of separation). BFS guarantees finding nodes at a specific "distance" (level) before moving to deeper levels, making it ideal for this kind of proximity-based search.

#### Application 2: Advanced PYMK with Scoring (DFS-like / Graph Analytics)

In a real-world system, a simple "friends of friends" might suggest too many irrelevant people. LinkedIn uses far more sophisticated algorithms that blend graph traversal with scoring and ranking. This often involves concepts from **DFS** and specialized graph algorithms, sometimes implicitly through iterative calculations.

**Factors for Scoring (Edge Weights and Node Attributes):**

*   **Common Connections:** The more mutual 2nd-degree connections, the higher the score.
    
*   **Shared Company/Industry:** Users from the same company or industry get a boost.
    
*   **Shared Skills/Endorsements:** Similar skill sets.
    
*   **Shared Groups/Interests:** Membership in the same groups.
    
*   **Activity Level:** More active users might be prioritized.
    

**How it works (Conceptual DFS/Graph Analytics):**

Instead of just finding _any_ 2nd-degree connection, LinkedIn's algorithms might:

1.  **Traverse the graph (DFS or other graph algorithms):** Explore paths from the current user.
    
2.  **Collect features:** For each potential connection (e.g., a 2nd-degree connection), gather data about shared attributes, common 1st-degree connections, etc.
    
3.  **Score potential connections:** Use machine learning models trained on vast amounts of user data to assign a "relevance score" to each potential connection. This model takes all the collected features as input.
    
4.  **Rank and recommend:** Present the top-N highest-scoring suggestions.
    

**Example: Counting Common Neighbors (A component of scoring):**

This isn't a full DFS, but it demonstrates how traversal knowledge (who are my neighbors, who are their neighbors) combines with counting.

In [10]:
def count_common_neighbors(user1, user2, connections_graph):
    """Counts the number of common direct connections between two users."""
    connections1 = connections_graph.get(user1, set())
    connections2 = connections_graph.get(user2, set())
    return len(connections1.intersection(connections2))

def get_people_you_may_know_scored(current_user, connections_graph, top_n=5):
    """
    A more advanced PYMK concept: finds friends of friends and scores them
    based on the number of common connections.
    """
    potential_suggestions = defaultdict(int) # {user_id: common_connections_count}
    direct_connections = connections_graph.get(current_user, set())

    # Find 2nd-degree connections and count common neighbors
    for first_degree_friend in direct_connections:
        for potential_friend in connections_graph.get(first_degree_friend, set()):
            if potential_friend != current_user and potential_friend not in direct_connections:
                # This is a 2nd-degree connection not directly connected to current_user
                potential_suggestions[potential_friend] += 1 # Simple increment for common 1st-degree friend

    # In a real system, you'd calculate a more complex score here
    # For simplicity, we're just using the count of common first-degree friends

    # Sort suggestions by score (descending)
    sorted_suggestions = sorted(potential_suggestions.items(), key=lambda item: item[1], reverse=True)

    return [user for user, score in sorted_suggestions[:top_n]]

# --- Test the scored PYMK ---
print(f"\nPeople Alice may know (scored by common friends): {get_people_you_may_know_scored('Alice', user_connections)}")
# Alice -> Bob (connected to David, Eve, Frank)
# Alice -> Charlie (connected to Frank)
#
# Potential from Bob: David (1 common with Alice: Bob), Eve (1 common with Alice: Bob), Frank (1 common with Alice: Bob)
# Potential from Charlie: Frank (1 common with Alice: Charlie)
#
# Resulting counts:
# David: 1 (via Bob)
# Eve: 1 (via Bob)
# Frank: 2 (via Bob, via Charlie)
#
# Sorted: Frank, David, Eve


People Alice may know (scored by common friends): ['Frank', 'David', 'Eve']


#### Application 3: Connection Path Analysis (DFS/BFS for specific paths)

Beyond just PYMK, LinkedIn might use graph algorithms to show the "path" between you and another user, for instance, showing "You know Mark via Bob and Sarah." This is a classic shortest path problem (if weighted by number of degrees) or simply a path-finding problem.

**How it works (BFS for shortest path):**

To find the shortest path (minimum degrees of separation), BFS is ideal. You can modify the BFS algorithm to store the parent of each visited node. Once the target node is found, you backtrack from the target to the source using these parent pointers.


In [11]:
def find_path(start_node, end_node, connections_graph):
    """
    Finds a shortest path (in terms of degrees of separation) between two users.
    Returns a list of users representing the path, or None if no path exists.
    """
    if start_node == end_node:
        return [start_node]

    queue = deque([(start_node, [start_node])]) # (current_user, current_path)
    visited = {start_node}

    while queue:
        current_user, path = queue.popleft()

        for neighbor in connections_graph.get(current_user, set()):
            if neighbor == end_node:
                return path + [end_node] # Found the target!
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor])) # Extend path and enqueue

    return None # No path found

# --- Test path finding ---
path_alice_heidi = find_path("Alice", "Heidi", user_connections)
print(f"\nPath from Alice to Heidi: {path_alice_heidi}") # Expected: ['Alice', 'Bob', 'Eve', 'Heidi'] or ['Alice', 'Charlie', 'Frank', 'Heidi']
# BFS will find one of the shortest paths.
# Let's verify our graph: Alice-Bob-Eve-Heidi (3 degrees)
# Alice-Charlie-Frank-Heidi (3 degrees)
# The output depends on the order of neighbors in sets/lists, but length should be shortest.

path_alice_grace = find_path("Alice", "Grace", user_connections)
print(f"Path from Alice to Grace: {path_alice_grace}") # Expected: ['Alice', 'Bob', 'David', 'Grace']


Path from Alice to Heidi: ['Alice', 'Bob', 'Frank', 'Heidi']
Path from Alice to Grace: ['Alice', 'Bob', 'David', 'Grace']


### Summary for LinkedIn and Graphs:

*   **Core Problem:** Modeling and leveraging user connections.
    
*   **Data Structure:** Adjacency List (using defaultdict(set) in Python for efficiency) due to the massive size and sparsity of the graph.
    
*   **Algorithms:**
    
    *   **BFS-like approaches:** For finding "friends of friends" (2nd-degree connections) and determining shortest paths (degrees of separation).
        
    *   **DFS-like explorations / Graph Analytics:** For collecting attributes for scoring potential connections, finding complex patterns, and possibly for identifying cycles (though less common in a direct social graph connection problem).
        
    *   Beyond BFS/DFS, real systems use advanced techniques like **Graph Embeddings** (representing nodes as vectors in a lower-dimensional space), **Graph Neural Networks (GNNs)**, and distributed graph processing frameworks (e.g., Apache Giraph, Neo4j, TinkerPop) to handle the scale and complexity.
        

This deep dive into LinkedIn's PYMK should give you a very concrete understanding of how fundamental graph theory and algorithms are to modern software development, especially in data-intensive and relationship-heavy applications.